In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from bs4 import BeautifulSoup
import concurrent.futures
import json
import time
import math

# [Keep all helper functions (get_summary_value, get_ld_data, get_data) unchanged] 
def get_summary_value(card, data_summary):
    element = card.select_one(f"[data-summary='{data_summary}']")
    if element:
        text = element.get_text(strip=True)
        if not text and element.next_sibling:
            text = element.next_sibling.strip() if isinstance(element.next_sibling, str) else ""
        return text
    return ""

def get_ld_data(card, idx):
    ld_json_data = {}
    ld_script = card.find("script", type="application/ld+json")
    if ld_script and ld_script.string:
        try:
            ld_json_data = json.loads(ld_script.string)
        except Exception as e:
            print(f"Error parsing ld+json for property {idx}: {e}")
    return ld_json_data

def get_data(soup):
    possible_classes = ['mb-srp__card', 'mb-srp_list', 'mb-srpCard', 'srpCard', 'mb-srp']
    css_selector = ', '.join([f'div.{cls}' for cls in possible_classes])
    cards = soup.select(css_selector)
    
    if not cards:
        print("No cards found with known class names.")
        return []
    
    properties = []
    for idx, card in enumerate(cards, 1):
        try:
            title = ''
            for title_class in ['mb-srp__card--title', 'mb-srp_card--title']:
                title_elem = card.find('h2', class_=title_class)
                if title_elem:
                    title = title_elem.get('title', '').strip() or title_elem.get_text(strip=True)
                    break
            
            price = ''
            for price_class in ['mb-srp__card__price--amount', 'mb-srp_card_price--amount']:
                price_elem = card.find('div', class_=price_class)
                if price_elem:
                    price = price_elem.get_text(strip=True)
                    break
            
            area = get_summary_value(card, 'super-area') or get_summary_value(card, 'carpet-area')
            transaction = get_summary_value(card, 'transaction')
            furnishing  = get_summary_value(card, 'furnishing')
            society     = get_summary_value(card, 'society')
            bathroom    = get_summary_value(card, 'bathroom')
            balcony     = get_summary_value(card, 'balcony')
            
            data = {
                "title": title,
                "price": price,
                "area": area,
                "transaction": transaction,
                "furnishing": furnishing,
                "society": society,
                "bathroom": bathroom,
                "balcony": balcony
            }
            
            usp_items = card.select("div.mb-srp_card_usp--item")
            data["usp_details"] = [item.get_text(strip=True) for item in usp_items] if usp_items else []
            
            ld_json_data = get_ld_data(card, idx)
            if ld_json_data:
                data["numberOfRooms"] = ld_json_data.get("numberOfRooms", "")
                geo_data = ld_json_data.get("geo", {})
                data["latitude"] = geo_data.get("latitude", "")
                data["longitude"] = geo_data.get("longitude", "")
            
            properties.append(data)
            print(f"Processed property {idx}: {title}")
            
        except Exception as e:
            print(f"Error on property {idx}: {str(e)}")
            continue
    return properties

def process_city_pages(city, start_page, end_page, retries=3):
    chrome_options = Options()
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
    
    driver = webdriver.Chrome(options=chrome_options)
    city_results = []
    
    try:
        for page in range(start_page, end_page + 1):
            url = (f"https://www.magicbricks.com/property-for-sale/residential-real-estate?"
                   f"proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment,"
                   f"Residential-House,Villa&page={page}&cityName={city}")
            
            for attempt in range(retries):
                try:
                    print(f"{city}: Page {page} (Attempt {attempt+1}/{retries})")
                    driver.get(url)
                    WebDriverWait(driver, 15).until
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.mb-srp__card, div.m-srp_card"))
                    
                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    props = get_data(soup)
                    city_results.extend(props)
                    print(f"{city}: Page {page} → {len(props)} properties")
                    break
                except Exception as e:
                    print(f"{city} Page {page} error: {str(e)}")
                    time.sleep(2)
    finally:
        driver.quit()
    return city_results

if __name__ == "__main__":
    city_property_counts = {
    "Noida": 12216,
    "Ghaziabad": 8216,
    "Greater-Noida": 8691,
    "Navi-Mumbai": 12357,
    "Faridabad": 3551,
    "Bhubaneswar": 3819,
    "Bokaro-Steel-City": 85,
    "Vijayawada": 888,
    "Vrindavan": 338,
    "Bhopal": 1996,
    "Gorakhpur": 154,
    "Jamshedpur": 1057,
    "Agra": 1170,
    "Allahabad": 589,
    "Jodhpur": 379,
    "Aurangabad": 594,
    "Jaipur": 10846,
    "Mangalore": 785,
    "Nagpur": 3502,
    "Guntur": 407,
    "Navsari": 90,
    "Palghar": 470,
    "Salem": 181,
    "Haridwar": 428,
    "Durgapur": 363,
    "Madurai": 561,
    "Manipal": 23,
    "Patna": 1389,
    "Ranchi": 1356,
    "Raipur": 870,
    "Sonipat": 616,
    "Kottayam": 154,
    "Kozhikode": 284,
    "Thrissur": 733,
    "Tirupati": 300,
    "Trivandrum": 853,
    "Trichy": 477,
    "Udaipur": 373,
    "Vapi": 179,
    "Varanasi": 925,
    "Vadodara": 4344,
    "Visakhapatnam": 2805,
    "Surat": 3556,
    "Kanpur": 1531,
    "Kochi": 1546,
    "Mysore": 804,
    "Goa": 2348,
    "Bhiwadi": 884,
    "Lucknow": 9189,
    "Nashik": 1692,
    "Guwahati": 1660,
    "Chandigarh": 2920,
    "Indore": 3525,
    "Coimbatore": 4075,
    "Dehradun": 2226
}

    all_properties = []
    MAX_WORKERS = 6  # Reduced for 16GB RAM stability
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {}
        for city, count in city_property_counts.items():
            end_page = math.ceil(count / 30)  # Calculate pages dynamically
            futures[executor.submit(process_city_pages, city, 1, end_page)] = city
        
        for future in concurrent.futures.as_completed(futures):
            city = futures[future]
            try:
                data = future.result()
                all_properties.extend(data)
                print(f"✅ {city}: Collected {len(data)} properties")
            except Exception as e:
                print(f"❌ {city} failed: {str(e)}")

    print(f"\nTotal collected: {len(all_properties)} properties")
    
    with open("properties.json", "w", encoding="utf-8") as f:
        json.dump(all_properties, f, indent=2, ensure_ascii=False)

converted the json file into excel


In [ ]:
import json
import pandas as pd

# Define input JSON file and output Excel file
json_file = "new.json"  # Change this to your actual file
excel_file = "output.xlsx"

# Step 1: Read and Fix JSON Format
try:
    with open(json_file, "r", encoding="utf-8") as file:
        raw_data = file.read()
    
    # Attempt to parse as a full JSON structure
    try:
        data = json.loads(raw_data)  # Load JSON
    except json.JSONDecodeError:
        print("⚠ JSON format error detected! Trying line-by-line parsing...")
        data = []
        with open(json_file, "r", encoding="utf-8") as file:
            for i, line in enumerate(file, 1):
                line = line.strip()
                if not line:
                    continue  # Skip empty lines
                try:
                    obj = json.loads(line)  # Parse each line
                    data.append(obj)
                except json.JSONDecodeError as e:
                    print(f"❌ Skipping line {i} due to JSON error: {e}")
        
    # Ensure valid data was read
    if not data:
        raise ValueError("No valid JSON objects found!")

except FileNotFoundError:
    print("❌ JSON file not found! Check the file path.")
    exit()
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    exit()

# Step 2: Save the formatted JSON (optional)
formatted_json_file = "formatted_total.json"
with open(formatted_json_file, "w", encoding="utf-8") as file:
    json.dump(data, file, indent=4)

print(f"✅ Formatted JSON saved to {formatted_json_file}")

# Step 3: Convert JSON to Pandas DataFrame
df = pd.DataFrame(data)

# Step 4: Convert list-type columns (like 'usp_details') to strings
for col in df.columns:
    df[col] = df[col].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)

# Step 5: Save to Excel
df.to_excel(excel_file, index=False, engine="openpyxl")

print(f"✅ Successfully converted JSON to {excel_file}")
